In [1]:
# To avoid warnings printed during the execution
import warnings
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import random
from sklearn.ensemble import AdaBoostClassifier

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
bigdata = pd.read_csv('bigdata(tf-idf).csv')

In [4]:
bigdata.head()

,Unnamed: 0,index,genre,bookId,entity.n.01,cheerfulness.n.01,eidos.n.01,ethos.n.01,inheritance.n.04,personality.n.01,...,deodorant.n.01,hairdressing.n.01,lotion.n.01,perfume.n.02,weaponry.n.01,ammunition.n.01,armament.n.01,gunnery.n.01,hardware.n.01,totalWords
0,0,0,1,14976,0.0,0.000000,0.0,0.0,0.000000,0.539379,...,0.0,0.0,0.0,0.0,0.0,1.488439,0.0,0.0,0.0,24106
1,1,1,1,2750,0.0,0.000000,0.0,0.0,0.171278,0.096741,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,44801
2,2,2,1,2754,0.0,0.000000,0.0,0.0,0.188879,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,20313
3,3,3,1,2747,0.0,0.606649,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.392177,0.0,0.0,0.0,18298
4,4,4,1,2749,0.0,0.000000,0.0,0.0,0.000000,0.148352,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,29215


In [5]:
bigdata = bigdata.drop(bigdata.loc[bigdata['genre'] == 2].index)
bigdata = bigdata.drop(bigdata.loc[bigdata['genre'] == 4].index)
bigdata = bigdata.drop(bigdata.loc[bigdata['genre'] == 8].index)
bigdata = bigdata.drop(bigdata.loc[bigdata['genre'] == 9].index)

In [6]:
bigdata.reset_index().head()

,level_0,Unnamed: 0,index,genre,bookId,entity.n.01,cheerfulness.n.01,eidos.n.01,ethos.n.01,inheritance.n.04,...,deodorant.n.01,hairdressing.n.01,lotion.n.01,perfume.n.02,weaponry.n.01,ammunition.n.01,armament.n.01,gunnery.n.01,hardware.n.01,totalWords
0,0,0,0,1,14976,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.488439,0.0,0.0,0.0,24106
1,1,1,1,1,2750,0.0,0.000000,0.0,0.0,0.171278,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,44801
2,2,2,2,1,2754,0.0,0.000000,0.0,0.0,0.188879,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,20313
3,3,3,3,1,2747,0.0,0.606649,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.392177,0.0,0.0,0.0,18298
4,4,4,4,1,2749,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,29215


In [7]:
msk = np.random.rand(len(bigdata)) < 0.8
train = bigdata[msk]
test = bigdata[~msk]

In [8]:
train.shape
test.shape

(530, 779)

In [9]:
temp = bigdata

In [10]:
genre = temp['genre']

In [11]:
del temp['genre']
del temp['Unnamed: 0']
del temp['index']
del temp['bookId']
del temp['totalWords']

In [12]:
Words = temp.values
# Words

In [66]:
pca = PCA(n_components=200)
ModelPca = pca.fit(Words)
pca

PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [67]:
# print pca.explained_variance_ratio_.sum()

In [68]:
df=pd.DataFrame()
for i in range(pca.explained_variance_ratio_.shape[0]):
    df["pc%i" % (i+1)] = Words[:,i]
df.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,...,pc191,pc192,pc193,pc194,pc195,pc196,pc197,pc198,pc199,pc200
0,0.0,0.000000,0.0,0.0,0.000000,0.539379,5.596673,2.739262,0.986586,0.094257,...,16.709685,0.0,0.0,0.535027,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.171278,0.096741,7.928731,6.593808,0.694189,0.101434,...,18.095715,0.0,0.0,0.000000,0.233771,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.188879,0.000000,6.053220,4.448405,0.180124,0.000000,...,7.279311,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.606649,0.0,0.0,0.000000,0.000000,8.026438,4.083569,1.399720,0.372527,...,5.480163,0.0,0.0,0.000000,0.572367,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,0.148352,6.663877,7.197050,0.688816,0.155548,...,4.421328,0.0,0.0,0.000000,0.716972,0.0,0.0,0.0,0.0,0.0


In [69]:
bigdata.head()

,entity.n.01,cheerfulness.n.01,eidos.n.01,ethos.n.01,inheritance.n.04,personality.n.01,property.n.02,quality.n.01,shape.n.02,space.n.01,...,cream.n.03,deodorant.n.01,hairdressing.n.01,lotion.n.01,perfume.n.02,weaponry.n.01,ammunition.n.01,armament.n.01,gunnery.n.01,hardware.n.01
0,0.0,0.000000,0.0,0.0,0.000000,0.539379,5.596673,2.739262,0.986586,0.094257,...,0.0,0.0,0.0,0.0,0.0,0.0,1.488439,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.171278,0.096741,7.928731,6.593808,0.694189,0.101434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.188879,0.000000,6.053220,4.448405,0.180124,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.606649,0.0,0.0,0.000000,0.000000,8.026438,4.083569,1.399720,0.372527,...,0.0,0.0,0.0,0.0,0.0,0.0,0.392177,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,0.148352,6.663877,7.197050,0.688816,0.155548,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [70]:
X_train, X_test, y_train, y_test = train_test_split(df, genre, test_size=0.20, random_state=42)

In [71]:
X_train.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,...,pc191,pc192,pc193,pc194,pc195,pc196,pc197,pc198,pc199,pc200
1431,0.0,0.000000,0.0,0.0,0.000000,0.0,10.671306,1.447783,0.000000,1.419807,...,0.708016,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
662,0.0,0.000000,0.0,0.0,0.000000,0.0,4.290866,0.000000,0.000000,0.000000,...,7.117226,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1602,0.0,0.000000,0.0,0.0,0.000000,0.0,8.414816,3.605189,1.897753,3.535526,...,1.763064,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1607,0.0,0.000000,0.0,0.0,0.000000,0.0,8.424738,1.785921,0.376040,0.000000,...,3.493512,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1858,0.0,0.250559,0.0,0.0,0.051961,0.0,5.936324,5.114715,0.396421,0.030772,...,9.445000,0.0,0.0,0.0,0.0,0.072004,0.0,0.0,0.0,0.0


In [81]:
rf =AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),
                         algorithm="SAMME",
                         n_estimators=2000)
rf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=2000, random_state=None)

In [82]:
pre=rf.predict(X_test)
accuracy_score(y_test, pre)

0.92883211678832112

In [83]:
pre=rf.predict(X_train)
accuracy_score(y_train, pre)

0.99406392694063928